# 1D Ising model

We consider the Hamiltonian

$$
\mathcal{H} = -\sum_i S_i S_{i+1}.
$$

We are going to update $S_i$.
We consider two states where $S_i=\pm 1$ and the other spins are the same as the current state.
The energy difference of these two states can be computed as $\Delta E = E_\uparrow - E_\downarrow = - 2h$,
where $h = S_{i-1} + S_{i+1}$.
We adopt $S_i=1$ with a probability of $1/(1+\exp(\beta \Delta E ))$.


In [1]:
@show VERSION
using BenchmarkTools, Random

VERSION = v"1.5.2"


## First simple implementation

In [2]:
function ising1d!(s, β, niters, rng)
    n = length(s)
    min_h = -2
    max_h = 2
    prob = [1/(1+exp(-2*β*h)) for h in min_h:max_h]
    for iter in 1:niters, i in 1:n
        sl = s[ifelse(i == 1, n, i-1)]
        sr = s[ifelse(i == n, 1, i+1)]
        # h = -2, 0, 2
        h = sl + sr
        si_old = s[i]
        s[i] = ifelse(rand(rng) < prob[h-min_h+1], +1, -1)
    end
end

n = 100
rng = MersenneTwister(4649)
s0 = rand(rng, Int8[-1, 1], n)
β = 100.0
niters = 10^3

s = copy(s0)

# Run once to compile the function
ising1d!(s, β, niters, rng)

@time ising1d!(s, β, niters, rng)
@benchmark ising1d!(s, β, niters, rng) setup=(s = copy(s0))

  0.000436 seconds (1 allocation: 128 bytes)


BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  1
  --------------
  minimum time:     414.421 μs (0.00% GC)
  median time:      433.919 μs (0.00% GC)
  mean time:        468.099 μs (0.00% GC)
  maximum time:     1.398 ms (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

## More complex implementation with measurement

In [3]:
# Wrap everything with a module to allow redefition of type
module MC

"""
Composite type to represent a spin state
"""
mutable struct SpinState
    num_spins::Int
    s::Array{Int8,1}
    energy::Int
    tot_mag::Int
end

"""
Energy
"""
function energy(s)
    n = length(s)
    - sum((s[i] * s[ifelse(i == n, 1, i+1)] for i in 1:n))
end

"""
Total magnetization
"""
total_magnetization(s) = sum(s)

"""
Constructor
"""
function SpinState(s)
    ss = SpinState(length(s), copy(s), energy(s), total_magnetization(s))
    sanity_check(ss)
    ss
end

"""
Sanity check
"""
function sanity_check(ss)
    @assert energy(ss.s) == ss.energy
    @assert total_magnetization(ss.s) == ss.tot_mag
end

"""
Take an object of SpinState as an input and update it in place.
"""
function update!(ss, β, niters, rng)
    min_h = -2
    max_h = 2
    s = ss.s
    n = ss.num_spins
    prob = [1/(1+exp(-2*β*h)) for h in min_h:max_h]
    for iter in 1:niters, i in 1:n
        sl = s[ifelse(i == 1, n, i-1)]
        sr = s[ifelse(i == n, 1, i+1)]
        # h = -2, 0, 2
        h = sl + sr
        si_old = s[i]
        s[i] = ifelse(rand(rng) < prob[h-min_h+1], +1, -1)
        
        # Update observables with O(1) operations
        ss.energy += (si_old - s[i]) * h
        ss.tot_mag += (s[i] - si_old)
    end
end

end
;

OK, create an updater object and run it, and check if a spin state is updated correctly.

In [4]:
ss = MC.SpinState(s0)
MC.update!(ss, β, niters, rng)
MC.sanity_check(ss)

In [5]:
@benchmark MC.update!(ss, β, niters, rng)

BenchmarkTools.Trial: 
  memory estimate:  128 bytes
  allocs estimate:  1
  --------------
  minimum time:     442.097 μs (0.00% GC)
  median time:      496.197 μs (0.00% GC)
  mean time:        510.269 μs (0.00% GC)
  maximum time:     1.161 ms (0.00% GC)
  --------------
  samples:          9795
  evals/sample:     1

## Exercise
Implement measurement of the magnetization and specific heat 
and compare the results with the exact ones!
(To do) Include analytic expressions